In [32]:
import os
import json
import re
import pandas as pd
import traceback

In [33]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

import PyPDF2
llm = Ollama(model="llama2")  

In [34]:
llm

Ollama()

In [35]:
RESPONSE_JSON = {
    "1": { "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"},
    "2": { "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"},
    "3": { "mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"},
}

In [36]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response as a valid JSON object ONLY, matching the RESPONSE_JSON below.
Ensure to make {number} MCQs.
RESPONSE_JSON:
{response_json}
"""

In [37]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [38]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [39]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
If the quiz is not at per with the cognitive and analytical abilities of the students, update the quiz questions which need to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [40]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [41]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [42]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [43]:
file_path="/Users/nirajgupta/mcqgen/data.txt"

In [44]:
file_path

'/Users/nirajgupta/mcqgen/data.txt'

In [45]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [46]:
print(TEXT)

Machine Learning (ML) is a branch of Artificial Intelligence (AI) that focuses on 
enabling computers to learn from data and make predictions or decisions without 
being explicitly programmed. Instead of writing detailed rules for every possible 
scenario, in ML we provide the system with large amounts of data and allow it to 
automatically learn patterns and relationships. This makes ML especially powerful 
for solving complex problems where rule-based programming would be impractical, 
such as image recognition, speech processing, or natural language understanding. 
The field of ML is generally divided into three main types: supervised learning, 
unsupervised learning, and reinforcement learning. In supervised learning, the
algorithm learns from labeled data, where each input example has a correct output
(label), allowing the system to learn a mapping from inputs to outputs. This is 
commonly used in tasks like predicting house prices, classifying emails as spam or
not, or detecting 

In [47]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [48]:
generate_evaluate_chain.verbose = False
response = generate_evaluate_chain({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Machine Learning (ML) is a branch of Artificial Intelligence (AI) that focuses on 
enabling computers to learn from data and make predictions or decisions without 
being explicitly programmed. Instead of writing detailed rules for every possible 
scenario, in ML we provide the system with large amounts of data and allow it to 
automatically learn patterns and relationships. This makes ML especially powerful 
for solving complex problems where rule-based programming would be impractical, 
such as image recognition, speech processing, or natural language understanding. 
The field of ML is generally divided into three main types: supervised learning, 
unsupervised learning, and reinforcement learning. In supervised learning, the
algorithm learns from labeled data, where each input example has a correct output
(label), allowing the system to learn a mapping from inputs to outputs. This is 
commonly used in tasks like predicting house prices, classifying email

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text:Machine Learning (ML) is a branch of Artificial Intelligence (AI) that focuses on 
enabling computers to learn from data and make predictions or decisions without 
being explicitly programmed. Instead of writing detailed rules for every possible 
scenario, in ML we provide the system with large amounts of data and allow it to 
automatically learn patterns and relationships. This makes ML especially powerful 
for solving complex problems where rule-based programming would be impractical, 
such as image recognition, speech processing, or natural language understanding. 
The field of ML is generally divided into three main types: supervised learning, 
unsupervised learning, and reinforcement learning. In supervised learning, the
algorithm learns from labeled data, where each input example has a correct output
(label), allowing the system to learn a mapping from inputs to outputs. This is 
commonly used in tasks like predicting house prices

In [49]:
print(response)

{'text': 'Machine Learning (ML) is a branch of Artificial Intelligence (AI) that focuses on \nenabling computers to learn from data and make predictions or decisions without \nbeing explicitly programmed. Instead of writing detailed rules for every possible \nscenario, in ML we provide the system with large amounts of data and allow it to \nautomatically learn patterns and relationships. This makes ML especially powerful \nfor solving complex problems where rule-based programming would be impractical, \nsuch as image recognition, speech processing, or natural language understanding. \nThe field of ML is generally divided into three main types: supervised learning, \nunsupervised learning, and reinforcement learning. In supervised learning, the\nalgorithm learns from labeled data, where each input example has a correct output\n(label), allowing the system to learn a mapping from inputs to outputs. This is \ncommonly used in tasks like predicting house prices, classifying emails as spam 

In [50]:
quiz=response.get("quiz")

In [51]:
if isinstance(quiz, str):
    match = re.search(r'\{.*\}', quiz, re.DOTALL)
    if match:
        quiz_json_str = match.group(0)
        try:
            quiz = json.loads(quiz_json_str)
        except Exception as e:
            print("JSON parsing error:", e)
            print("Extracted string:", quiz_json_str)
    else:
        print("Could not find JSON object in quiz string.")


In [52]:
print("Type of quiz:", type(quiz))
print("Value of quiz:", quiz)

Type of quiz: <class 'dict'>
Value of quiz: {'1': {'mcq': 'What is machine learning?', 'options': ['A branch of Artificial Intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed.', 'A type of programming language used for creating artificial intelligence systems.', 'The process of training a computer model to perform a specific task.'], 'correct': 'A branch of Artificial Intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed.'}, '2': {'mcq': 'What are the three main types of machine learning?', 'options': ['Supervised learning, unsupervised learning, and reinforcement learning.', 'Unsupervised learning, supervised learning, and deep learning.', 'Reinforcement learning, supervised learning, and unsupervised learning.'], 'correct': 'Supervised learning, unsupervised learning, and reinforcement learning.'}, '3': {'mcq': 'What is the typical machine le

In [53]:
quiz_table_data = []
if isinstance(quiz, dict):
    for key, value in quiz.items():
        mcq = value.get("mcq", "")
        options = value.get("options", {})

        if isinstance(options, dict):
            options_str = " | ".join(
                [f"{opt}: {opt_value}" for opt, opt_value in options.items()]
            )
        elif isinstance(options, list):
            options_str = " | ".join(
                [f"{chr(97 + i)}: {opt}" for i, opt in enumerate(options)]
            )
        else:
            options_str = ""

        correct = value.get("correct", "")
        quiz_table_data.append({"MCQ": mcq, "Choices": options_str, "Correct": correct})
else:
    print("Error: 'quiz' should be a dictionary.")


In [54]:
for row in quiz_table_data:
    print(row)

{'MCQ': 'What is machine learning?', 'Choices': 'a: A branch of Artificial Intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed. | b: A type of programming language used for creating artificial intelligence systems. | c: The process of training a computer model to perform a specific task.', 'Correct': 'A branch of Artificial Intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed.'}
{'MCQ': 'What are the three main types of machine learning?', 'Choices': 'a: Supervised learning, unsupervised learning, and reinforcement learning. | b: Unsupervised learning, supervised learning, and deep learning. | c: Reinforcement learning, supervised learning, and unsupervised learning.', 'Correct': 'Supervised learning, unsupervised learning, and reinforcement learning.'}
{'MCQ': 'What is the typical machine learning workflow?', 'Choices': 'a: Collecting and pre

In [55]:
df = pd.DataFrame(quiz_table_data)

In [56]:
print(df)

                                                 MCQ  \
0                          What is machine learning?   
1  What are the three main types of machine learn...   
2     What is the typical machine learning workflow?   

                                             Choices  \
0  a: A branch of Artificial Intelligence that en...   
1  a: Supervised learning, unsupervised learning,...   
2  a: Collecting and preprocessing data, training...   

                                             Correct  
0  A branch of Artificial Intelligence that enabl...  
1  Supervised learning, unsupervised learning, an...  
2  Collecting and preprocessing data, training a ...  


In [57]:
output_folder = "/Users/nirajgupta/mcqgen/output"
os.makedirs(output_folder, exist_ok=True)

In [58]:
csv_file_path = os.path.join(output_folder, "mcq_quiz.csv")
df.to_csv(csv_file_path, index=False)

In [59]:
print(f"CSV file saved at: {csv_file_path}")

CSV file saved at: /Users/nirajgupta/mcqgen/output/mcq_quiz.csv
